## Imports and Setup

In [1]:
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import os 
os.chdir('../')
from src import sat_download
from utils import data_utils
from utils import config_utils
from utils import pred_utils

%load_ext autoreload
%autoreload 2

/anaconda/envs/gigav3/lib/python3.10/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


## Load Config Files

In [9]:
iso_code="TJK"
adm_level="ADM2"
sum_threshold=5
buffer_size=150
spacing=150

config_file = os.path.join(os.getcwd(), "configs/data_configs/data_config_ISO_AS.yaml")
sat_config_file = os.path.join(os.getcwd(), "configs/sat_configs/sat_config_500x500_60cm.yaml")
sat_creds_file = os.path.join(os.getcwd(), "configs/sat_configs/issa_sat_creds.yaml")

config = config_utils.load_config(config_file)
sat_config = config_utils.load_config(sat_config_file)
sat_creds = config_utils.create_config(sat_creds_file)

## Generate Prediction Tiles

In [ ]:
tiles = pred_utils.batch_process_tiles(
    config, iso_code, spacing, buffer_size, sum_threshold, adm_level
)

## Download Satellite Images

In [ ]:
pred_utils.batch_download_sat_images(
    sat_config, sat_creds, config, iso_code, spacing, buffer_size, sum_threshold, adm_level
)